# **Ensembles para Regressão — Módulo 4, Notebook Extra 2**

---

## Índice

1. [Introdução ao Ensembles](#introducao)
2. [Bagging](#bagging)
3. [Formalização Matemática](#formalizacao)
4. [Random Forest](#random-forest)
5. [Gradient Boosting](#gradient-boosting)
6. [Combinando Modelos Diferentes](#voting-stacking)
7. [Entendendo Bias vs Variância](#bias-variancia)
8. [Comparando Todos os Modelos](#comparacao)
9. [Quando Usar Cada Método](#quando-usar)
10. [Considerações Importantes](#consideracoes)

---

<a id='introducao'></a>
## **Introdução ao Ensembles**

Você já parou para pensar na sabedoria coletiva? Quando você quer tomar uma grande decisão, frequentemente consulta várias pessoas. Um conselho de especialistas tende a tomar decisões melhores do que um especialista isolado.

O mesmo princípio aplica-se a machine learning. **Ensembles** são algoritmos que combinam múltiplos modelos para fazer previsões. A ideia central é simples: múltiplos modelos, cada um cometendo erros de forma ligeiramente diferente, quando combinados, tendem a dar melhores resultados.

Você já aprendeu sobre **Regressão Linear**, **SVR** e **Árvores de Decisão** para regressão. Todos têm suas forças e fraquezas. Agora vamos descobrir como juntá-los de forma inteligente!

### **A Situação Prática**

Imagine que você precisa prever o **preço de casas** na Califórnia. Você tem dados sobre:
- Onde a casa fica (latitude/longitude)
- Quantos quartos tem em média
- A renda das pessoas na região
- Quão velhas são as construções

Se cada modelo fosse uma pessoa dando uma opinião:

- **Regressão Linear**: "É simples, casa maior = maior preço, relação é direta"
- **SVR**: "Depende do contexto, às vezes o preço sobe rápido, às vezes devagar"
- **Árvore de Regressão**: "Se a casa está em tal região com tantos quartos, o preço é X, senão é Y"

**Ensembles fazem exatamente isso:** juntam essas interpretações complementares para chegar a uma previsão melhor. Em vez de votar em classes (como em classificação), agora fazemos a **média de predições numéricas**.

<a id='bagging'></a>
## **Bagging (Bootstrap Aggregating)**

Se você leu o capítulo de Ensembles para Classificação, já sabe como Bagging funciona. A mecânica é idêntica, mas em vez de **votar** ("8 de 10 árvores dizem que sobreviveu"), agora fazemos **média**:

### **Os Três Passos do Bagging**

1. **Pega** o dataset de treino
2. **Cria** várias versões dele usando bootstrap (amostragem com reposição)
3. **Treina** um modelo em cada versão
4. **Combina** as previsões fazendo a média

Exemplo prático:
- Árvore 1 prevê: $2.5M
- Árvore 2 prevê: $2.8M
- Árvore 3 prevê: $2.3M
- **Previsão final**: média = $(2.5 + 2.8 + 2.3) / 3 = 2.53M$

### **Por que Funciona?**

Cada árvore erra de um jeito diferente. Uma superestima o preço, outra subestima. Quando você tira a média, esses erros tendem a se cancelar mutuamente, reduzindo a variância geral.

### **Detalhe Importante: OOB Score**

Lembre-se que ~37% dos dados ficam de fora de cada bootstrap (Out-of-Bag). Podemos usar esses dados para avaliar o modelo **de graça**, sem precisar separar um conjunto de validação! Em regressão, o OOB score é basicamente o R² calculado nessas amostras não vistas.

### **Vendo Isso na Prática**

Vamos usar o **California Housing Dataset** (preços de casas na Califórnia) para observar como Bagging melhora a performance.

### Usando o BaggingRegressor do Sklearn

Agora vamos usar a implementação pronta do sklearn. Ativaremos `oob_score=True` para obter o "score grátis" nos dados Out-of-Bag, funcionando como um conjunto de validação sem precisar separar dados!

In [ ]:
# =======================================================
# IMPORTS INICIAIS
# =======================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Carregar dataset California Housing
california = fetch_california_housing()
X_full = pd.DataFrame(california.data, columns=california.feature_names)
y_full = california.target  # MedHouseVal

# Usar subconjunto para velocidade (opcional)
X = X_full.sample(n=6000, random_state=42)
y = y_full[X.index]

X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size=0.25, random_state=42)

In [ ]:
# =======================================================
# Árvore única de regressão (baseline)
# =======================================================
base_tree = DecisionTreeRegressor(random_state=42, max_depth=None)
base_tree.fit(X_train, y_train)

pred_base = base_tree.predict(X_test)
rmse_base = mean_squared_error(y_test, pred_base)
mae_base = mean_absolute_error(y_test, pred_base)
r2_base = r2_score(y_test, pred_base)

print(f"Árvore Única - RMSE: {rmse_base:.3f} | MAE: {mae_base:.3f} | R²: {r2_base:.3f}")

# Visualização real vs predito
plt.scatter(y_test, pred_base, alpha=0.4)
plt.xlabel('Real')
plt.ylabel('Predito')
plt.title('Árvore Única - Real vs Predito')
lims = [min(y_test.min(), pred_base.min()), max(y_test.max(), pred_base.max())]
plt.plot(lims, lims, 'r--')
plt.show()


In [ ]:
# =======================================================
# Várias Árvores bootstrap (manual) para observar variância
# =======================================================
B = 12 
metrics = []
for b in range(B):
    indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
    X_boot = X_train[indices]
    y_boot = y_train[indices]
    tree_b = DecisionTreeRegressor(random_state=b)
    tree_b.fit(X_boot, y_boot)
    pred_b = tree_b.predict(X_test)
    r2_b = r2_score(y_test, pred_b)
    rmse_b = mean_squared_error(y_test, pred_b)
    metrics.append((r2_b, rmse_b))
    print(f"Árvore {b+1:02d} -> R²: {r2_b:.3f} | RMSE: {rmse_b:.3f}")

r2_vals = [m[0] for m in metrics]
print(f"\nR² Médio: {np.mean(r2_vals):.3f} | Desvio: {np.std(r2_vals):.3f} | Min: {min(r2_vals):.3f} | Max: {max(r2_vals):.3f}")

plt.figure(figsize=(8,4))
plt.plot(r2_vals, marker='o')
plt.axhline(np.mean(r2_vals), color='red', linestyle='--', label='Média R²')
plt.title('Distribuição de R² entre Árvores Bootstrap')
plt.xlabel('Árvore')
plt.ylabel('R²')
plt.legend()
plt.show()


<a id='formalizacao'></a>
## **Formalização Matemática**

Agora que você entendeu intuitivamente como os ensembles funcionam, vamos formalizar os conceitos matematicamente.

### **Bagging**

A predição do Bagging é a média simples das predições de todos os modelos treinados:

$$\hat{y}_{\text{bag}}(x) = \frac{1}{B}\sum_{b=1}^{B}f_b(x)$$

Onde:
- $B$: número de modelos (árvores)
- $f_b(x)$: predição do modelo $b$ para a entrada $x$
- $\hat{y}_{\text{bag}}(x)$: predição final do ensemble

### **Out-of-Bag Score**

Para cada observação no conjunto de treino, aproximadamente 37% não foram selecionadas em cada bootstrap. Podemos usar essas amostras para calcular o R² sem necessidade de separar um conjunto de validação:

$$R^2_{\text{OOB}} = 1 - \frac{\sum_{i=1}^{n}(y_i - \hat{y}_{OOB,i})^2}{\sum_{i=1}^{n}(y_i - \bar{y})^2}$$

### **Gradient Boosting**

Gradient Boosting é sequencial: cada modelo novo tenta corrigir os erros do anterior. A predição é feita acumulativamente:

$$F_m(x) = F_{m-1}(x) + \eta f_m(x)$$

Onde:
- $F_{m-1}(x)$: predição acumulada até o modelo anterior
- $f_m(x)$: predição do modelo novo (que tenta prever os resíduos)
- $\eta$: learning rate (taxa de aprendizado) que controla o peso de cada correção
- $F_m(x)$: predição final acumulada

A ideia é treinar cada novo modelo para prever os **resíduos** do anterior:

$$r_{i,m} = y_i - F_{m-1}(x_i)$$

E depois adicionar a predição deste modelo à anterior.

### **Random Forest**

Random Forest combina Bagging com seleção aleatória de features. A predição é similar ao Bagging, mas cada árvore considera apenas um subset aleatório de features em cada divisão, aumentando a diversidade:

$$\hat{y}_{\text{RF}}(x) = \frac{1}{B}\sum_{b=1}^{B}f_b(x) \quad \text{onde cada } f_b \text{ usa features aleatórias}$$

<a id='random-forest'></a>
## **Random Forest**

In [ ]:
from sklearn.ensemble import BaggingRegressor

bag = BaggingRegressor(
    estimator=DecisionTreeRegressor(random_state=0),
    n_estimators=50,
    oob_score=True,
    random_state=42,
    n_jobs=-1
)
bag.fit(X_train, y_train)

pred_bag = bag.predict(X_test)
rmse_bag = mean_squared_error(y_test, pred_bag)
mae_bag = mean_absolute_error(y_test, pred_bag)
r2_bag = r2_score(y_test, pred_bag)
print(f"Bagging - RMSE: {rmse_bag:.3f} | MAE: {mae_bag:.3f} | R² Teste: {r2_bag:.3f} | OOB R²: {bag.oob_score_:.3f}")

Random Forest é um Bagging que além de criar versões diferentes dos dados (bootstrap), também escolhe **features aleatórias** em cada divisão da árvore.

### **Por que Isso?**

No Bagging normal, se uma feature é muito forte (ex: "renda mediana" em preços de casas), **todas** as árvores vão usá-la na primeira divisão. Resultado? As árvores ficam muito parecidas (correlacionadas) e a redução de variância é menor.

Random Forest força cada árvore a considerar apenas um **subset aleatório de features** em cada divisão. Isso deixa as árvores mais diferentes umas das outras, e quando você tira a média, ganha ainda mais estabilidade.

### **Quando Usar Random Forest**

Random Forest costuma ser o **primeiro modelo que você deve tentar** em problemas com dados tabulares (tabelas). Funciona bem out-of-the-box e raramente falha de forma catastrófica.

### **Hiperparâmetros Principais**

- `n_estimators`: quantas árvores treinar (mais = melhor, até um ponto de saturação)
- `max_features`: quantas features considerar por split (default `'sqrt'` funciona bem na prática)
- `max_depth`: profundidade máxima de cada árvore (controla complexidade)
- `min_samples_leaf`: número mínimo de amostras em uma folha (evita sobreajuste)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

pred_rf = rf.predict(X_test)
rmse_rf = mean_squared_error(y_test, pred_rf)
mae_rf = mean_absolute_error(y_test, pred_rf)
r2_rf = r2_score(y_test, pred_rf)
print(f"RandomForest - RMSE: {rmse_rf:.3f} | MAE: {mae_rf:.3f} | R²: {r2_rf:.3f}")

# Importância de features
imp = rf.feature_importances_
cols = california.feature_names
order = np.argsort(imp)[::-1]
print("\nImportância das features:")
for i in order:
    print(f"{cols[i]:15}: {imp[i]:.4f}")

plt.figure(figsize=(7,4))
plt.bar([cols[i] for i in order], imp[order])
plt.xticks(rotation=45)
plt.ylabel('Importância')
plt.title('Random Forest - Importância de Features')
plt.tight_layout()
plt.show()


<a id='gradient-boosting'></a>
## **Gradient Boosting**

Bagging e Random Forest são **paralelos**: todas as árvores são treinadas independentemente, cada uma vendo um subset de dados.

Gradient Boosting é **sequencial**: cada árvore nova tenta **corrigir os erros** da anterior, refinando a predição iterativamente.

### **A Ideia Fundamental**

1. Treina a primeira árvore (faz uma predição inicial dos preços)
2. Calcula os **resíduos** (erros: real - previsto)
3. Treina a segunda árvore **para prever os resíduos**, não os valores originais
4. Adiciona essa correção à predição anterior
5. Calcula novos resíduos
6. Repete até ter $N$ árvores

É como ir refinando a estimativa aos poucos. Cada árvore é fraca (pequena, com poucas divisões), mas juntas formam um modelo extremamente forte.

### **Learning Rate (Taxa de Aprendizado)**

O parâmetro $\eta$ (eta) controla o "peso" de cada correção. 

- Se for muito alto (ex: $\eta = 0.5$), você aprende rápido mas pode overfitar facilmente
- Se for muito baixo (ex: $\eta = 0.01$), você precisa de muitas árvores mas generaliza melhor

**Regra de bolso:** learning_rate baixo (~0.05) + muitas árvores (~300) costuma funcionar bem.

### **Early Stopping**

Um recurso crítico do Gradient Boosting é o **early stopping**. Você treina monitorizando o desempenho em um conjunto de validação. Quando o desempenho para de melhorar (ou piora), você para de adicionar árvores.

```python
gb = GradientBoostingRegressor(
    learning_rate=0.05,
    n_estimators=1000,  # pode parecer alto, mas...
    max_depth=3,
    random_state=42,
    validation_fraction=0.2,  # 20% para validação
    n_iter_no_change=10  # para se não melhorar em 10 iterações
)
gb.fit(X_train, y_train)
```

Isso previne overfitting automático e economiza tempo computacional.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

lrs = [0.05, 0.1, 0.2]
results_gb = []
for lr in lrs:
    gb = GradientBoostingRegressor(
        learning_rate=lr,
        n_estimators=300,
        max_depth=3,
        random_state=42
    )
    gb.fit(X_train, y_train)
    pred_gb = gb.predict(X_test)
    r2_gb = r2_score(y_test, pred_gb)
    rmse_gb = mean_squared_error(y_test, pred_gb)
    results_gb.append((lr, r2_gb, rmse_gb))
    print(f"GB (lr={lr}) -> R²: {r2_gb:.3f} | RMSE: { rmse_gb:.3f}")

plt.figure(figsize=(6,4))
plt.plot([r[0] for r in results_gb],[r[1] for r in results_gb], marker='o')
plt.xlabel('learning_rate')
plt.ylabel('R²')
plt.title('Gradient Boosting - efeito do learning_rate')
plt.show()


<a id='voting-stacking'></a>
## **Combinando Modelos Diferentes: Voting e Stacking**

Até agora combinamos várias **versões do mesmo modelo** (várias árvores). E se combinarmos **modelos completamente diferentes**?

### **VotingRegressor: Média Simples**

Você treina vários modelos diferentes (Linear, SVR, RandomForest) e faz a **média simples** das predições.

Exemplo:
- Regressão Linear prevê: $2.3M (captura tendência geral linear)
- SVR prevê: $2.7M (captura não-linearidades)
- Random Forest prevê: $2.5M (captura interações complexas)
- **Voting**: $(2.3 + 2.7 + 2.5) / 3 = 2.5M$

**Vantagem:** Cada modelo contribui sua "expertise" complementar. A média suaviza outliers individuais.

### **StackingRegressor: Aprendendo a Combinar**

Em vez de só tirar a média, Stacking treina um **modelo meta** que aprende a melhor forma de combinar as predições. É como ter vários especialistas dando opinião e um coordenador inteligente que sabe quanto peso dar para cada um.

**Processo:**
1. Treina vários modelos base (Linear, SVR, RF) em subset dos dados
2. Faz predições desses modelos em outro subset
3. Usa essas predições como **features** para treinar um modelo meta (ex: Gradient Boosting)
4. O modelo meta aprende a melhor combinação

**Quando usar?** Quando você quer extrair os últimos 0.5-1% de performance. Mas cuidado: mais complexidade = mais chance de overfitting.

In [ ]:
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

lin = LinearRegression()
svr = SVR(kernel='rbf', C=10, epsilon=0.1)
small_rf = RandomForestRegressor(n_estimators=120, random_state=7, n_jobs=-1)

vote = VotingRegressor([
    ('lin', lin),
    ('svr', svr),
    ('rf', small_rf)
])
vote.fit(X_train, y_train)
pred_vote = vote.predict(X_test)
print(f"VotingRegressor R²: {r2_score(y_test, pred_vote):.3f}")

stack = StackingRegressor(
    estimators=[('lin', LinearRegression()), ('svr', SVR(kernel='rbf', C=10, epsilon=0.1)), ('rf', RandomForestRegressor(n_estimators=150, random_state=11))],
    final_estimator=GradientBoostingRegressor(learning_rate=0.05, n_estimators=200, max_depth=3, random_state=42),
    passthrough=False,
    n_jobs=-1
)
stack.fit(X_train, y_train)
pred_stack = stack.predict(X_test)
print(f"StackingRegressor R²: {r2_score(y_test, pred_stack):.3f}")

<a id='bias-variancia'></a>
## **Entendendo Bias vs Variância com Ensembles**

Lembra do dilema **bias-variância** que aprendemos? É crucial para entender por que ensembles funcionam.

- **Bias alto**: modelo é simples demais, não captura padrões (underfitting)
- **Variância alta**: modelo muda muito com pequenas alterações nos dados (overfitting)

### **Como Cada Ensemble Reduz Variância e Bias**

| Modelo | Bias | Variância | Abordagem |
|--------|------|-----------|-----------|
| **Árvore única profunda** | Baixo ✅ | Alta ❌ | Captura tudo, mas instável |
| **Bagging** | Baixo ✅ | Baixa ✅ | Média estabiliza predições |
| **Random Forest** | Baixo ✅ | Muito Baixa ✅✅ | Bagging + diversidade |
| **Gradient Boosting** | Muito Baixo ✅✅ | Média ⚠️ | Reduz bias iterativamente |

### **A Estratégia de Cada Ensemble**

**Bagging / Random Forest:**
- Reduz **variância** drasticamente (média estabiliza)
- Mantém bias relativamente baixo
- **Resultado:** Melhor para a maioria dos casos

**Gradient Boosting:**
- Reduz **bias** progressivamente (corrige erros de forma sequencial)
- Se exagerar, pode aumentar variância (overfit)
- **Resultado:** Precisa de tuning cuidadoso, mas consegue performance muito alta

### **Diagnóstico: Como Saber Se Está Overfitting?**

Se seu modelo está overfitting (treino ótimo, teste ruim):

- ↓ `max_depth` (árvores mais rasas)
- ↑ `min_samples_leaf` (folhas maiores)
- ↓ `learning_rate` (boosting mais conservador)
- ↑ `min_samples_split` (divisões mais conservadoras)

Se seu modelo está underfitting (treino e teste ruins):

- ↑ `n_estimators` (mais árvores/iterações)
- ↑ `max_depth` (mais profundidade)
- ↑ `learning_rate` (aprender mais rápido)

<a id='comparacao'></a>
## **Comparando Todos os Modelos**

Agora que treinamos vários ensembles, vamos colocá-los lado a lado e ver quem performou melhor no nosso dataset de casas da Califórnia!

In [ ]:
# Garantir que já existem variáveis dos modelos (base_tree, bag, rf, results_gb, vote, stack)
# Pegar melhor Gradient Boosting (maior R²)
best_gb_lr, best_gb_r2, best_gb_rmse = sorted(results_gb, key=lambda x: x[1], reverse=True)[0]

comparacao = []
# Regressor base
comparacao.append(['Árvore Única', r2_base, rmse_base])
comparacao.append(['Bagging', r2_bag, rmse_bag])
comparacao.append(['RandomForest', r2_rf, rmse_rf])
comparacao.append([f'GradientBoosting(lr={best_gb_lr})', best_gb_r2, best_gb_rmse])
comparacao.append(['Voting', r2_score(y_test, pred_vote), mean_squared_error(y_test, pred_vote)])
comparacao.append(['Stacking', r2_score(y_test, pred_stack), mean_squared_error(y_test, pred_stack)])

res_df = pd.DataFrame(comparacao, columns=['Modelo','R2','RMSE']).sort_values('R2', ascending=False)
print(res_df)

plt.figure(figsize=(7,4))
plt.bar(res_df['Modelo'], res_df['R2'])
plt.xticks(rotation=40, ha='right')
plt.ylabel('R²')
plt.title('Comparação de Modelos - Ensembles Regressão')
plt.tight_layout()
plt.show()


<a id='quando-usar'></a>
## **Quando Usar Cada Método**

Agora que você conhece todos os ensembles, como escolher qual usar?

### **Tabela de Decisão Prática**

| Cenário | Recomendação | Razão |
|---------|--------------|-------|
| **Primeiro modelo a testar** | Random Forest | Melhor out-of-the-box, sem tuning |
| **Dados pequenos (<1000 amostras)** | Random Forest | Bagging funciona bem com poucos dados |
| **Dados grandes (>100k amostras)** | Gradient Boosting ou LightGBM | Mais control fine-tuning permite melhor performance |
| **Features correlacionadas** | Gradient Boosting | GB lida melhor com redundância |
| **Precisa interpretabilidade** | Random Forest + Feature Importance | RF é mais fácil de explicar |
| **Busca máxima performance** | Gradient Boosting + Stacking | Estes conseguem os últimos % de melhoria |
| **Computação limitada** | Random Forest | Menos intensivo que GB/Stacking |
| **Dados desbalanceados** | Gradient Boosting | Pode ajustar class_weight |

### **Seu Workflow Prático**

1. **Baseline rápido**: Testa um RandomForest com configurações padrão
2. **Tuning**: Ajusta `n_estimators`, `max_depth`, `min_samples_leaf`
3. **Upgrade**: Experimenta GradientBoosting com learning_rate baixo e early stopping
4. **Ensemble heterogêneo**: Se ainda não tá bom, combina modelos diferentes com Voting/Stacking
5. **Próximo nível**: Explora XGBoost, LightGBM ou CatBoost (versões turbinadas)

<a id='consideracoes'></a>
## **Considerações Importantes**

### **1. Custo Computacional**

Ensembles são **significativamente mais lentos** que modelos únicos. Você está treinando dezenas ou centenas de árvores. Para datasets grandes, isso importa:

- **Random Forest**: Pode paralelizar facilmente (`n_jobs=-1`)
- **Gradient Boosting**: Sequencial por natureza, mas mais rápido por árvore
- **Stacking**: Muito lento (treina múltiplos modelos base + modelo meta)

### **2. Interpretabilidade vs Performance**

Ensembles sacrificam interpretabilidade para ganhar performance:

- ❌ Difícil explicar "por que o modelo decidiu isso"
- ✅ Use feature importance para entender quais features importam
- ✅ Use SHAP values para explicações mais detalhadas
- ⚠️ Em contextos regulatórios (ex: crédito), isso pode ser crítico

### **3. Overfitting em Gradient Boosting**

Gradient Boosting agressivo pode overfitar facilmente:

- ✅ Use **early stopping** para monitorar validação
- ✅ Mantenha `learning_rate` baixo (~0.05)
- ✅ Mantenha árvores rasas (`max_depth=3` a `max_depth=5`)
- ✅ Aumente `min_samples_leaf`

### **4. Quando Ir para XGBoost/LightGBM**

Se você chegou ao máximo com Gradient Boosting sklearn, a próxima evolução natural é:

- **XGBoost**: Versão otimizada de Gradient Boosting com regularização
- **LightGBM**: Mais rápido, usa growth by leaf (menos profundo)
- **CatBoost**: Excelente com features categóricas

Esses modelos ganham competições de Kaggle, mas a base teórica é a mesma que você aprendeu aqui!

---

<-- [**Anterior: Árvores de Decisão para Regressão**](03_arvores_regressao.ipynb) | [**Próximo: Módulo 5: Introdução**](../05_Unsupervised_Learning/01_introducao.ipynb) -->